In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from umap.umap_ import UMAP
from sklearn.preprocessing import StandardScaler

2023-10-25 19:03:30.294475: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
users = pd.read_json('babynamesDB_users.json')
users['num_groups'] = [len(i) for i in users['groups']]
users = users[['_id' , 'num_comments_stored', 'groups', 'num_posts_stored', 'num_groups']]
users = users.explode('groups')
users = pd.concat([users, pd.get_dummies(users['groups'], dtype=float)], axis=1)
users = users.groupby('_id').sum()

In [3]:
reducer = UMAP()
scaled_data = StandardScaler().fit_transform(users)
embedding = reducer.fit_transform(scaled_data)

In [ ]:
'''
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sns.color_palette()[x] for x in users['num_groups']]
    )
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the BabyNamesDB dataset', fontsize=24)
'''